In [1]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time
from pathlib import Path
import pandas as pd
pc = Pinecone(api_key="pcsk_368Tpa_ULGYwn74G43mBVq1hEUaaReNWtM38AFjwhU3cpJjKvpXqaV4GxJDGWaY12tagMR")

In [4]:

# # Initialize a Pinecone client with your API key
# pc = Pinecone(api_key="pcsk_368Tpa_ULGYwn74G43mBVq1hEUaaReNWtM38AFjwhU3cpJjKvpXqaV4GxJDGWaY12tagMR")

# # Define a sample dataset where each item has a unique ID and piece of text
# data = [
#     {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
#     {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
#     {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
#     {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
#     {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
#     {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
# ]

# # Convert the text into numerical vectors that Pinecone can index
# embeddings = pc.inference.embed(
#     model="multilingual-e5-large",
#     inputs=[d['text'] for d in data],
#     parameters={"input_type": "passage", "truncate": "END"}
# )

# print(embeddings)


In [5]:
# Get the folder where this script is located
df = pd.read_csv("schrute.csv", header=0)
df = df.drop("Unnamed: 0", axis=1)
print(df.columns)

df['lower'] = df['text'].str.lower()
df['lower'] = df['lower'].str.strip()
# Replace empty strings with NaN and then drop those rows
df.replace(pd.NA, "", inplace=True)

Index(['index', 'season', 'episode', 'episode_name', 'director', 'writer',
       'character', 'text', 'text_w_direction'],
      dtype='object')


In [6]:
arr = df['text'].tolist()
idx = df['index'].tolist()

In [7]:
# Create a serverless index
index_name = "the-office"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)


In [8]:
all_embeddings = []
for i in range(0, 1496+1, 96):
    embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=arr[i:i+96],
    parameters={"input_type": "passage", "truncate": "END"}
    )
    all_embeddings.append(embeddings)

In [71]:
# for i in range(0, 1496+1, 96):
#     print(i, i+96)
#     for j in range(i, i+96+1):
#         print(j)

In [10]:
# Target the index where you'll store the vector embeddings

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'

records = []
w = 0
for i in range(0, 1496+1, 96):
    k = 0
    for j in range(i, i+96):
        records.append({
            "id": str(df['index'].loc[j].tolist()),
            "values": all_embeddings[0][k]['values'],
            "metadata": {'text': df['text'].loc[j]}
        })
        k += 1
    w += 1


In [11]:
len(all_embeddings), len(all_embeddings[0]), len(all_embeddings[-1])

(16, 96, 96)

In [12]:
len(records)

1536

In [3]:

index = pc.Index("the-office")


In [16]:
for i in range(0, 1496+1, 96):
    # Upsert the records into the index
    index.upsert(
        vectors=records[i:i+96],
        namespace="example-namespace"
    )

In [5]:
# Define your query
query = " I might just be a basketball machine."

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)


{'matches': [{'id': '141',
              'metadata': {'text': 'Introduce yourself. Be polite.'},
              'score': 0.7969082,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '237',
              'metadata': {'text': "Mr. Decker, I'm sorry about that. What "
                                   'were you... Can you hold on one second? '
                                   'Yeah, just one second. Thanks. Hello? '
                                   "That's it. Perfect. So what I was "
                                   'saying... Hello? Thanks, Dwight.'},
              'score': 0.7969082,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '45',
              'metadata': {'text': 'OK...'},
              'score': 0.7969082,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': 'example-namespace',
 'usage': {'read_un

In [ ]:
results['matches'][0]['id'],

{'id': '429',
 'metadata': {'text': 'We can go.'},
 'score': 0.81124663,
 'sparse_values': {'indices': [], 'values': []},
 'values': []}

In [22]:
int(results['matches'][0]['id'])

429

[428, 812, 908]